# ODC - Original data analysis
This notebook looks at the data as it is currently store in ODC and compares it against observed data

# Notebook set-up

## 1. Setting Python Modules

In [1]:
%matplotlib inline

import sys
from pathlib import Path
from datetime import datetime as dt
from datetime import timedelta
import re
import pandas as pd
from osgeo import gdal, osr
import xarray as xr
import rioxarray
import numpy as np
from pprint import pprint

# Plot library
import h5py
import holoviews as hv
import matplotlib.pyplot as plt
# from jupyterlab_h5web import H5Web
from IPython.display import display, HTML
from bokeh.io import output_notebook
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show

output_notebook() # this is need so bokeh display in notebook


AttributeError: module 'osgeo._gdalconst' has no attribute 'DCAP_FLUSHCACHE_CONSISTENT_STATE'

In [2]:
import dask.dataframe as dd
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 30.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43695,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 30.73 GiB
Comm: tcp://127.0.0.1:46653,Total threads: 2
Dashboard: http://127.0.0.1:45447/status,Memory: 7.68 GiB
Nanny: tcp://127.0.0.1:42133,


## Setting global vars

In [2]:
catch_path = Path('/home/geofelpave/Documents/6_Test_Area/data/catchments_ukbn2/')

## 2. Process

### Working with Precipitation data

In [3]:
# Read shp file to geopandas dataframe
grid_path = Path(catch_path / 'UKBN2_eigthy_py_nrfa_testLakes_v2_wgs84.shp')
grid = gpd.read_file(grid_path)

xmin, ymin, xmax, ymax = grid.total_bounds

# Get data
# Load data from the datacube
buffer = 0.125
START_DATE = "1980-01-01"
END_DATE = "1982-12-31"

# Load data from the datacube
ds_rain = dc.load(product='era5_reanalysis_tp_daily',
             lat=(ymin - buffer, ymax + buffer),
             lon=(xmin - buffer, xmax + buffer),
             time=(START_DATE, END_DATE),
             dask_chunks={'time': 1, 'longitude': 200, 'latitude': 200},
             )

ds_rain


<xarray.Dataset>
Dimensions:              (time: 1096, latitude: 6, longitude: 5)
Coordinates:
  * time                 (time) datetime64[ns] 1980-01-01T00:29:59.500000 ......
  * latitude             (latitude) float64 55.65 55.55 55.45 55.35 55.25 55.15
  * longitude            (longitude) float64 -2.75 -2.65 -2.55 -2.45 -2.35
    spatial_ref          int32 4326
Data variables:
    total_precipitation  (time, latitude, longitude) float32 dask.array<chunksize=(1, 6, 5), meta=np.ndarray>
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

In [4]:
ds_rain = ds_rain.resample(time='Y').mean()
# ds_rain

# ds_rain = ds_rain.resample(time='Y').mean('time').mean(dim='time')
# ds_rain

In [5]:
   # Create list of timestamps
lst_time = ds_rain.coords['time'].values
lst_time

array(['1980-12-31T00:00:00.000000000', '1981-12-31T00:00:00.000000000',
       '1982-12-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
for x in range(len(lst_time)):
    xr_rain = ds_rain.isel(time=slice(x, x + 1))
    # df_rain = xr_rain.to_dask_dataframe()
    # print(df_rain['total_precipitation'].sum().compute())

In [6]:
import rasterstats

test = ds_rain.isel(time=slice(0, 0 + 1))
wrk_path = Path('/home/geofelpave/Documents/6_Test_Area/data/temp/')
_tif = Path(wrk_path / f'precipitation.tif')
test['total_precipitation'].rio.to_raster(_tif)
# catchm_stats_mean = rasterstats.zonal_stats(str(Path(catch_path / 'UKBN2_eigthy_py_nrfa_testLakes_v2_wgs84.shp')), 
#                                             str(_tif), stats="sum", 
#                                             all_touched=True, nodata=-9999)

# catchm_stats_mean

/home/geofelpave/anaconda3/envs/cubo-ubuntu/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


In [45]:
test = ds_rain.isel(time=slice(0, 0 + 1))
new = test.to_dask_dataframe()


In [49]:
new['total_precipitation'].sum().compute()

/home/geofelpave/anaconda3/envs/cubo-ubuntu/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


78.32556

In [ ]:
loaded_ds = ds_rain.compute()
client.close()
loaded_ds

In [35]:
client.close()

2023-12-06 16:17:26,067 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('dc_load_total_precipitation-getitem-mean_chunk-132194a1111636ea9a4214e463ab78b9', 4, 0, 0))" coro=<Worker.execute() done, defined at /home/geofelpave/anaconda3/envs/cubo-ubuntu/lib/python3.10/site-packages/distributed/worker_state_machine.py:3610>> ended with CancelledError
2023-12-06 16:17:26,067 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('dc_load_total_precipitation-getitem-mean_chunk-132194a1111636ea9a4214e463ab78b9', 0, 0, 0))" coro=<Worker.execute() done, defined at /home/geofelpave/anaconda3/envs/cubo-ubuntu/lib/python3.10/site-packages/distributed/worker_state_machine.py:3610>> ended with CancelledError
2023-12-06 16:17:26,067 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('dc_load_total_precipitation-getitem-mean_chunk-132194a1111636ea9a4214e463ab

In [31]:
df_precip = df.groupby(['time'])['total_precipitation'].sum().to_frame().reset_index()
df_precip

,time,total_precipitation
0,1980-12-31,78.325546
1,1981-12-31,71.320862
2,1982-12-31,77.993393


In [32]:
TOOLTIPS = [
    ("Precip", "@total_precipitation"),
    ("date", "@time"),
]

p = figure(width=800, height=400, x_axis_type='datetime', tooltips=TOOLTIPS)
p.line(x='time', y='total_precipitation', source=df_precip, line_color='blue', legend_label="ODC")

show(p)